Notebook 1

# Set up

Import necessary libraries, upload the CSVs to google colab, and convert them into pandas dataframes

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import math

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Partition6467LinkData.csv to Partition6467LinkData.csv
Saving Partition6467ProbePoints.csv to Partition6467ProbePoints.csv


In [ ]:
probeCols = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading']
linkCols = ['linkPVID', 'refNodeID', 'nrefNodeID', 'length', 'functionalClass', 'directionOfTravel', 'speedCategory', 
            'fromRefSpeedLimit', 'toRefSpeedLimit', 'fromRefNumLanes', 'toRefNumLanes', 'multiDigitized', 'urban', 
            'timeZone', 'shapeInfo', 'curvatureInfo', 'slopeInfo']
matchCols = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading', 
             'linkPVID', 'direction', 'distFromRef', 'distFromLink']

In [ ]:
probePoints = pd.read_csv(io.BytesIO(uploaded['Partition6467ProbePoints.csv']), ',', header=None, names=probeCols)
linkData = pd.read_csv(io.BytesIO(uploaded['Partition6467LinkData.csv']), ',', header=None, names=linkCols)

# Parsing

Function to parse the columns that contain lists in LinkData

In [ ]:
def stof(x):
  if x is not '':
    return float(x)
  else:
    return np.NaN

def parseInfo(x):
  if x is not None and x is not np.NaN:
    a = x.split('|')
    z = []
    for y in a:
      z.append(tuple(map(stof, y.split('/'))))
    return z

Modify the dataframes with the parsing function

In [ ]:
linkData['shapeInfo'] = linkData['shapeInfo'].apply(parseInfo)
linkData['curvatureInfo'] = linkData['curvatureInfo'].apply(parseInfo)
linkData['slopeInfo'] = linkData['slopeInfo'].apply(parseInfo)

# Approach #1
## DO NOT ATTEMPT TO RUN

create the output dataframe

In [ ]:
matchedPoints = pd.DataFrame(columns=matchCols)

Distance from point to line segment
[geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors](geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors)

In [ ]:
def dist_to_link(A, B, E):
  '''
  calculate perpendicular distance between point E and line AB
  A, B, E = (lat, lon)
  '''
 
  # vector AB 
  AB = [None, None]; 
  AB[0] = B[0] - A[0]; 
  AB[1] = B[1] - A[1]; 
 
  # vector BP 
  BE = [None, None];
  BE[0] = E[0] - B[0]; 
  BE[1] = E[1] - B[1]; 
 
  # vector AP 
  AE = [None, None];
  AE[0] = E[0] - A[0];
  AE[1] = E[1] - A[1]; 
 
  # Variables to store dot product 
 
  # Calculating the dot product 
  AB_BE = AB[0] * BE[0] + AB[1] * BE[1]; 
  AB_AE = AB[0] * AE[0] + AB[1] * AE[1]; 
 
  # Minimum distance from 
  # point E to the line segment 
  reqAns = 0; 
 
  # Case 1 
  if (AB_BE > 0) :
 
      # Finding the magnitude 
      y = E[1] - B[1]; 
      x = E[0] - B[0]; 
      reqAns = math.sqrt(x * x + y * y); 
 
  # Case 2 
  elif (AB_AE < 0) :
      y = E[1] - A[1]; 
      x = E[0] - A[0]; 
      reqAns = math.sqrt(x * x + y * y); 
 
  # Case 3 
  else:
 
      # Finding the perpendicular distance 
      x1 = AB[0]; 
      y1 = AB[1]; 
      x2 = AE[0]; 
      y2 = AE[1]; 
      mod = math.sqrt(x1 * x1 + y1 * y1); 
      reqAns = abs(x1 * y2 - y1 * x2) / mod if mod != 0 else 0; 
     
  return reqAns;

distance formula

In [ ]:
def dist_to_point(A, B):
  '''
  calculate euclidean distance between point A and point B
  A = (lat, lon)
  B = (lat, lon)
  '''
  return math.sqrt((A[0] - B[0])**2 + (A[1] - B[1])**2)

`match` is a function that takes a probe point and iterates over the linkData dataframe using the distance_to_link formula to find the nearest link for the given probe point.

In [ ]:
def match(p):
  '''
  Given probe point p and the linkData df,
  determine which link to which p most likely belongs
  '''
  E = (p[4], p[5]) # coordinates of probe point

  n = None # the nearest link so far
  d = None # the perpendicular distance to link n

  for l in linkData.itertuples():

    y = l[15] # shape info
    A = (y[0][0], y[0][1]) # coordinates of ref node
    #B = (y[-1][0], y[-1][1]) # coordinates of nonref node
    dist = dist_to_point(A, E)

    if n is None or d is None or dist < d:
      d = dist
      n = l

  A = (n[15][0][0], n[15][0][1]) # coordinates of reference point
  d2 = dist_to_point(E, A)

  return (n,d,d2)

apply `match` to every probe point and put the results into the output dataframe

In [ ]:
i = 0
for p in probePoints.itertuples():
  print(i, end=", ")
  if(i % 100 == 0):
    print()
  i += 1

  (n,d,e) = match(p)
  row = {
    'sampleID': p[0],
    'dateTime': p[1],
    'sourceCode': p[2],
    'latitude': p[3],
    'longitude': p[4],
    'altitude': p[5],
    'speed': p[6],
    'heading': p[7],
    'linkPVID': n[0],
    'direction': n[5],
    'distFromRef': e,
    'distFromLink': d
  }
  matchedPoints.append(row, ignore_index=True)

# Approach #2 (original, buggy, DO NOT RUN)

## ideas
- try filtering link data by the direction of travel of the probe point
- track each sampleID (trajectory) separately

## find links incident to a given link L
get all links whose refNode matches the refNode or nonRefNode of L and get all links whose nonRefNode matches the refNode or nonRefNode of L.


inspired by [http://www.vldb.org/conf/2005/papers/p853-brakatsoulas.pdf](http://www.vldb.org/conf/2005/papers/p853-brakatsoulas.pdf)


## code

iterate over all probe points, adding their `sampleID`s to a python set so that we have a definite list of all of the trajectories from which points were collected

In [ ]:
# Find all unique SampleIDs (trajectories)
samples = probePoints['sampleID']
trajectories = set(())
for s in samples:
  trajectories.add(s)
print(len(trajectories))

75840


For each link in the road network, get a slice of the linkData dataframe consisting of all those links that have the original link's reference node as their reference node or non-reference node. This will be a list of all of the links to which the original link is incident. Add this list into a new column in the linkData dataframe.

In [ ]:
# For each road link, add a column with a list of links to which it is incident
i = 0
linkData['incident'] = None
for l in linkData.itertuples():
  if i % 500 == 0:
    print(i, end=', ')
  if i % 10000 == 0:
    print()
  i += 1

  refNode = l[2]
  nonRefNode = l[3]
  ilinks = linkData[(linkData.refNodeID == refNode) | 
                    (linkData.refNodeID == nonRefNode) | 
                    (linkData.nrefNodeID == refNode) | 
                    (linkData.nrefNodeID == nonRefNode)]
  linkData.at[l[0], 'incident'] = ilinks['linkPVID'].tolist()

Download the new linkData dataframe as a CSV

In [ ]:
# export new link data as CSV
linkData.to_csv('newLinkData.csv')
files.download('newLinkData.csv')

Create the output dataframe

In [ ]:
# create the output dataframe
matchedPoints = pd.DataFrame(columns=matchCols)

Distance from point to line segment [geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors](https://geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors)

In [ ]:
# distance from point to line
# geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors
def minDistance(A, B, E) : 
 
    # vector AB 
    AB = [None, None]; 
    AB[0] = B[0] - A[0]; 
    AB[1] = B[1] - A[1]; 
 
    # vector BP 
    BE = [None, None];
    BE[0] = E[0] - B[0]; 
    BE[1] = E[1] - B[1]; 
 
    # vector AP 
    AE = [None, None];
    AE[0] = E[0] - A[0];
    AE[1] = E[1] - A[1]; 
 
    # Variables to store dot product 
 
    # Calculating the dot product 
    AB_BE = AB[0] * BE[0] + AB[1] * BE[1]; 
    AB_AE = AB[0] * AE[0] + AB[1] * AE[1]; 
 
    # Minimum distance from 
    # point E to the line segment 
    reqAns = 0; 
 
    # Case 1 
    if (AB_BE > 0) :
 
        # Finding the magnitude 
        y = E[1] - B[1]; 
        x = E[0] - B[0]; 
        reqAns = math.sqrt(x * x + y * y); 
 
    # Case 2 
    elif (AB_AE < 0) :
        y = E[1] - A[1]; 
        x = E[0] - A[0]; 
        reqAns = math.sqrt(x * x + y * y); 
 
    # Case 3 
    else:
 
        # Finding the perpendicular distance 
        x1 = AB[0]; 
        y1 = AB[1]; 
        x2 = AE[0]; 
        y2 = AE[1]; 
        mod = math.sqrt(x1 * x1 + y1 * y1);
        reqAns = 0 if mod == 0 else abs(x1 * y2 - y1 * x2) / mod; 
     
    return reqAns; 

distance formula

In [ ]:
# calculate distance between two points
def distance(A, B):
  # Calculating distance 
  return math.sqrt(math.pow(B[0] - A[0], 2) +
                math.pow(B[1] - B[1], 2) * 1.0)

Iterate over all the _trajectories_ in the probePoints dataframe. For each trajectory (sampleID), get the slice of the probePoints that have that sampleID and sort it by time. This will get a sequence of coordinates in time. For the first point, iterate over all links to find the nearest. Then each subsequent point only needs to be matched against the links incident to one determined nearest for the previous point. This eliminates the need to check _every_ probe point against _every_ link, which drastically improves performance. Once the nearest link is determined, add the information as a row to the output dataframe.

In [ ]:
# iterate over each trajectory
i = 1
for t in trajectories:
  print("{}".format(t), end=', ')
  if i % 100 == 0:
    print()
  i += 1

  # get the slice of the df that is relevant to this trajectory (sampleID)
  singleProbe = probePoints[probePoints.sampleID == t]
  # sort this new df by the datetime column
  singleProbe.sort_values('dateTime')

  ituples = singleProbe.itertuples()
  nearestPoint = None
  nearestDist = None
  
  # for the first point, search all links for nearest
  firstPoint = next(ituples)
  for l in linkData.itertuples():
    ref = l[15][0]
    nonref = l[15][-1]
    p = (firstPoint[4], firstPoint[5])
    d = minDistance(ref, nonref, p)
    if nearestPoint is None or nearestDist is None or d < nearestDist:
      nearestPoint = l
      nearestDist = d

  # create and append the first row for this trajectory
  row = {
    'sampleID': firstPoint[1],
    'dateTime': firstPoint[2],
    'sourceCode': firstPoint[3],
    'latitude': firstPoint[4],
    'longitude': firstPoint[5],
    'altitude': firstPoint[6],
    'speed': firstPoint[7],
    'heading': firstPoint[8],
    'linkPVID': nearestPoint[1],
    'direction': nearestPoint[6],
    'distFromRef': distance((firstPoint[4], firstPoint[5]), l[15][0]),
    'distFromLink': nearestDist
  }
  matchedPoints.append(row, ignore_index=True)

  # for all subsequent points, search all links that
  # are incident to the previous for the nearst
  for point in ituples:
    incidentLinks = nearestPoint[18]
    for il in incidentLinks: # il is a linkPVID incident to nearestPoint
      l = linkData.loc[linkData['linkPVID'] == il]
      if len(l) != 1:
        print('multiple linkPVID {}'.format(il))
      l = next(l.itertuples())
      ref = l[15][0]
      nonref = l[15][-1]
      p = (point[4], point[5])
      d = minDistance(ref, nonref, p)
      if d < nearestDist:
        nearestPoint = l
        nearestDist = d

    # create and append the next row for this trajectory
    row = {
      'sampleID': point[1],
      'dateTime': point[2],
      'sourceCode': point[3],
      'latitude': point[4],
      'longitude': point[5],
      'altitude': point[6],
      'speed': point[7],
      'heading': point[8],
      'linkPVID': nearestPoint[1],
      'direction': nearestPoint[6],
      'distFromRef': distance((point[4], point[5]), l[15][0]),
      'distFromLink': nearestDist
    }
    matchedPoints.append(row, ignore_index=True)